## Pong 01

El siguiente programa muestra la primera versión de pong: 
Hemos definido una clae para la bola (`Ball`). La clase implementa los
mñetodos update (Por ahora solo se mueve hacia la derecha) y
`draw`, de forma que la responsabilidad de actualizar su posición
y la de pintarse en pantalla son asumidas por la propia bola:

In [11]:
import math
import pygame
import random
import time

SIZE = WIDTH, HEIGHT = 800, 640  # Tamaño de pantalla

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego

class Ball:
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
        self.speed = 10
        
    def update(self):
        self.x += self.speed
    
    def draw(self, canvas):
        rect = pygame.Rect((self.x-5, self.y-5), (11, 11))
        pygame.draw.rect(canvas, WHITE, rect)
        
pygame.init()
try:
    pygame.display.set_caption("Pong 01")
    screen = pygame.display.set_mode(SIZE, 0, 24)
    limits = screen.get_rect()
    
    # Parte de inicialización del juego
    ball = Ball(limits.centerx, limits.centery)
    clock = pygame.time.Clock()    
    in_game = True
    while in_game:
        # Obtener datos de entrada
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                in_game = False
        # Recalcular el estado del juego, en base al estado actual y a las entradas
        ball.update()
        # Representamos el nuevo estado
        screen.fill(BLACK)
        ball.draw(screen)
        pygame.display.update()
        clock.tick(FPS)
finally:
    pygame.quit()

### Vamos a cambiar el código para que se mueva en diagonal

El porblema es que si cambiamos el codigo del update, por ejemplo, a:
    
```Python
    class Ball:
        ...
        def update(self):
            self.x += self.speed
            self.y += self.speed
        ...
```

La velocidad de la bola no será la misma que cuando se movia en linea recta, porque 
ahora la bola no se desplaza 10 pixels cada frame, sino más. Podemos calcular cuanto más por
el teorema de pitagoras:
    
> Teorema de Pitágoras.- En un triángulo rectángulo, el cuadrado de la hipotenusa es igual a la suma de los cuadrados de los catetos.

Es decir, la distacia que viaja la pelota cada frame es igual a:

$$ d = \sqrt{10^2 + 10^2} = \sqrt{200} = 10\sqrt{2} \approx 14.142135623730951$$

Como podemos calcular facilmente usando la funcion `sqrt` (raíz cuadrada) del módulo `math`.

In [15]:
10 * math.sqrt(2)

14.142135623730951

![velocidad](velocidad.png)